In [1]:
import os
import glob
import random
import csv
import numpy as np
from PIL import Image
from torchvision.transforms import functional as F

In [2]:
basedir = '/projects/patho2/External_Data/ISIC/'
groundtruth = os.path.join(basedir, 'ISIC_2020_Training_GroundTruth.csv')

In [32]:
reader = csv.reader(open(groundtruth, "r"), delimiter=",")
x = list(reader)

In [33]:
im_list = []
label_list = []
for row in range(1, len(x)):
    im_list.append(x[row][0])
    label_list.append(int(x[row][-1]))

In [34]:
ind_random = list(range(len(im_list)))
random.shuffle(ind_random)
train_ind = ind_random[:int(len(ind_random) * 0.8)]
valid_ind = ind_random[int(len(ind_random) * 0.8):]

In [37]:
with open(os.path.join(basedir, 'train.txt'), 'a') as f:
    for t_ind in train_ind:
        im_bn = im_list[t_ind]
        im_name = os.path.join(basedir,'train', '{}.jpg'.format(im_bn))
        assert os.path.exists(im_name)
        label = label_list[t_ind]
        f.write('{};{}\n'.format(im_name, label))
        
    

In [38]:
with open(os.path.join(basedir, 'valid.txt'), 'a') as f:
    for v_ind in valid_ind:
        im_bn = im_list[v_ind]
        im_name = os.path.join(basedir,'train', '{}.jpg'.format(im_bn))
        assert os.path.exists(im_name)
        label = label_list[v_ind]
        f.write('{};{}\n'.format(im_name, label))
        
    

In [30]:
test_im = glob.glob(os.path.join(basedir, 'test', '*.jpg'))
with open(os.path.join(basedir, 'test.txt'), 'a') as f:
    for t_im in test_im:
        f.write('{};-1\n'.format(t_im))

In [20]:
with open(os.path.join(basedir, 'train.txt'), 'r') as f:
    image_list = [line.rstrip() for line in f]


In [25]:
# with open(os.path.join(basedir, 'train_new.txt'), 'w') as f:
#     for line in image_list:
#         im_p, label = line.split(';')
#         im_type = os.path.basename(os.path.dirname(im_p))
#         im_bn = os.path.basename(im_p)
#         image = Image.open(im_p)
#         crop_size = (4096, 6144)
#         paths = []
#         for scale in [0.25, 0.5, 0.75]:
#             scaled_path = os.path.join(basedir, im_type, 'x{}'.format(scale), im_bn)
#             paths.append(scaled_path)
#             scaled_size = [int(scale * c_size) for c_size in crop_size]
#             resized = F.resize(image, scaled_size, Image.BICUBIC)
#             resized.save(scaled_path)
#         f.write('{};{};{};{};{}\n'.format(paths[0], paths[1], paths[2], im_p, label))
        
# with open(os.path.join(basedir, 'valid.txt'), 'r') as f:
#     image_list = [line.rstrip() for line in f]
# with open(os.path.join(basedir, 'valid_new.txt'), 'w') as f:
#     for line in image_list:
#         im_p, label = line.split(';')
#         im_type = os.path.basename(os.path.dirname(im_p))
#         im_bn = os.path.basename(im_p)
#         image = Image.open(im_p)
#         crop_size = (4096, 6144)
#         paths = []
#         for scale in [0.25, 0.5, 0.75]:
#             scaled_path = os.path.join(basedir, im_type, 'x{}'.format(scale), im_bn)
#             paths.append(scaled_path)
#             scaled_size = [int(scale * c_size) for c_size in crop_size]
#             resized = F.resize(image, scaled_size, Image.BICUBIC)
#             resized.save(scaled_path)
#         f.write('{};{};{};{};{}\n'.format(paths[0], paths[1], paths[2], im_p, label))
with open(os.path.join(basedir, 'test.txt'), 'r') as f:
    image_list = [line.rstrip() for line in f]
with open(os.path.join(basedir, 'test_new.txt'), 'w') as f:
    for line in image_list:
        im_p, label = line.split(';')
        im_type = os.path.basename(os.path.dirname(im_p))
        im_bn = os.path.basename(im_p)
        image = Image.open(im_p)
        crop_size = (4096, 6144)
        paths = []
        for scale in [0.25, 0.5, 0.75]:
            scaled_path = os.path.join(basedir, im_type, 'x{}'.format(scale), im_bn)
            paths.append(scaled_path)
#             scaled_size = [int(scale * c_size) for c_size in crop_size]
#             resized = F.resize(image, scaled_size, Image.BICUBIC)
#             resized.save(scaled_path)
        for p in paths:
            assert os.path.exists(p), '{} does not exist'.format(p)
        f.write('{};{};{};{};{}\n'.format(paths[0], paths[1], paths[2], im_p, label))
        

In [22]:
import sys
sys.path.append('/projects/patho1/melanoma_diagnosis/code/multi-scale_melanoma_diagnosis/')
from dataset.transforms import Zooming
from torchvision.transforms import Compose
zoom_transform = Compose([
    Zooming(scale_levels=[0.25, 0.5, 0.75, 1.0], size=(4096, 6144))
])

In [ ]:
with open(os.path.join(basedir, 'train.txt'), 'r') as f:
    image_list = [line.rstrip() for line in f]
for line in image_list:
    im_p, label = line.split(';')
    im_type = os.path.basename(os.path.dirname(im_p))
    im_bn = os.path.basename(im_p)
    image = Image.open(im_p)
    crop_size = (4096, 6144)
    im_1x = F.resize(image, crop_size, Image.BICUBIC)
    
    im_1x.save(im_p)
    paths = []
    resized = zoom_transform({'image': im_1x, 'mask':None})
    for i, scale in enumerate(['0.25_2', '0.5_2', '0.75_2', '1.0_2']):
        scaled_path = os.path.join(basedir, im_type, 'x{}'.format(scale), im_bn)
        paths.append(scaled_path)
#             scaled_size = [int(scale * c_size) for c_size in crop_size]
#             resized = F.resize(image, scaled_size, Image.BICUBIC)
        resized['image'][i].save(scaled_path)
    with open(os.path.join(basedir, 'experiment_txt', 'transform2', 'train.txt'), 'a') as f:
        f.write('{};{};{};{};{}\n'.format(paths[0], paths[1], paths[2], paths[3], label))
        
        
with open(os.path.join(basedir, 'valid.txt'), 'r') as f:
    image_list = [line.rstrip() for line in f]
for line in image_list:
    im_p, label = line.split(';')
    im_type = os.path.basename(os.path.dirname(im_p))
    im_bn = os.path.basename(im_p)
    image = Image.open(im_p)
    crop_size = (4096, 6144)
    im_1x = F.resize(image, crop_size, Image.BICUBIC)
    im_1x.save(im_p)
    paths = []
    resized = zoom_transform({'image': im_1x, 'mask':None})
    for i, scale in enumerate(['0.25_2', '0.5_2', '0.75_2', '1.0_2']):
        scaled_path = os.path.join(basedir, im_type, 'x{}'.format(scale), im_bn)
        paths.append(scaled_path)
#             scaled_size = [int(scale * c_size) for c_size in crop_size]
#             resized = F.resize(image, scaled_size, Image.BICUBIC)
        resized['image'][i].save(scaled_path)
    with open(os.path.join(basedir, 'experiment_txt', 'transform2', 'valid.txt'), 'a') as f:
        f.write('{};{};{};{};{}\n'.format(paths[0], paths[1], paths[2], paths[3], label))
        

with open(os.path.join(basedir, 'test.txt'), 'r') as f:
    image_list = [line.rstrip() for line in f]
for line in image_list:
    im_p, label = line.split(';')
    im_type = os.path.basename(os.path.dirname(im_p))
    im_bn = os.path.basename(im_p)
    image = Image.open(im_p)
    crop_size = (4096, 6144)
    im_1x = F.resize(image, crop_size, Image.BICUBIC)
    im_1x.save(im_p)
    paths = []
    resized = zoom_transform({'image': im_1x, 'mask':None})
    for i, scale in enumerate(['0.25_2', '0.5_2', '0.75_2', '1.0_2']):
        scaled_path = os.path.join(basedir, im_type, 'x{}'.format(scale), im_bn)
        paths.append(scaled_path)
#             scaled_size = [int(scale * c_size) for c_size in crop_size]
#             resized = F.resize(image, scaled_size, Image.BICUBIC)
        resized['image'][i].save(scaled_path)
    with open(os.path.join(basedir, 'experiment_txt', 'transform2', 'test.txt'), 'a') as f: 
        f.write('{};{};{};{};{}\n'.format(paths[0], paths[1], paths[2], paths[3], label))